# Cynthia Kielpinski

## Research Interests:

By majoring in statistics, I have always had a passion for the process behind statistics. The collection of data, the wrangling and processing of the data, exploratory analysis of the data, interpreting the analysis, and then finalizing the process by formulating a conclusion from the analysis. One fundamental ideal in statistics I have learned is that without an interdiscplinary field, you do not have the data and you are simply performing pure statistics. As per this comment, I have never found interest in pure statistics thus I found a passion in the interdisciplinary field of ecology and environmental sciences. For this project, my research interests are focus on exploratory statistics (required) and an environmental or ecological factor. Environmental data is more prominent  than ecological from my persepctive thus I have decided to focus on an environmental influence. Since our group decided on usng a summer and winter Olympics dataset, I though to refine my first research question to involve different countries performance in the summer and winter olympics influenced by their "local" climate. Competition is always a prfound topic, and I was also interested in how countries perform in their home country versus a visiting country. 

#### These are my generalize research questions:
- climate and its relation to medals earned
- host city correlation to medals earned

## Research Questions: 

1. **Does the climate zone of a country denoted by the Köppen climate classification system determine a correlation between the probability or likelihood of preforming more succesfully in the summer or winter Olmypics?** For example, does Canada have a better probability of winning medals in the winter Olympics since it is in group D of the Köppen climate classification system (continental climate) defined specifically as subartic climate? Does Brazil have a better probability of winning medals in the summer Olympics since it is in group A of the Köppen climate classification system (tropical climates) defined as a tropical wet or savannah climate? Or is there no correlation between a countries climate classification and which Olympics the country preforms better in? I will explore the correlations between countries and their climates in regards to number of medals won in each of the respective summer or winter Olympic events. 
2. **By hosting the Olympics, does the hosting country win more medals in comparision to non-hosting countries?** I will explore the correlations to the country hosting the Olympics and the total number of medals won and compare the non-hosting countries total number of medals won. 


## Exploratory Data Analysis Plan: 

To determine some answers into my research questions, some of my ideas for exploratory analysis are: regression analysis, principal component analysis, analysis of variance and other simple but profound statistical methods. From briefly looking over the datasets, the above methods are a brief thought to the possibilities when exploring the data. After wrangling the data, I will define the specific analyses methods I choose to use. I have a background in Statistics, thus deciding what analyses will be used at a later stage will be an simplistic decision. 

I will complete as little wrangling as needed, but I have visioned the data to have a few more columns in regards to location, hosting city, average number of medals won by country, total number of medals won, total number of bronze medals, total number of silver medals, total number of gold medals, and anything data-wise that can be combined from the other datasets we have got approved. Additionally, I think I will merge the "three" datasets, and then remove the columns that aren't needed and remove the missing values if there are any present. Lastly, I think I will also rename the column names to make them easier for analysis.


## Import Packages: (all packages needed for analysis, updated 02-12-2022)

In [9]:
## Import Packages:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
## Data Upload:
OlympicHost = pd.read_csv('../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_hosts.csv')

OlympicMedals = pd.read_csv('../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_medals.csv')

## Have to create köppen climate classification csv!! Chatting to Dr. Moosvi.

## Initial Observations: 

In [11]:
OlympicHost.head(n=5)

,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014


In [12]:
OlympicMedals.head(n=5)

,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_url,athlete_full_name,country_name,country_code,country_3_letter_code
0,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/stefania-cons...,Stefania CONSTANTINI,Italy,IT,ITA
1,Curling,beijing-2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,https://olympics.com/en/athletes/amos-mosaner,Amos MOSANER,Italy,IT,ITA
2,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/kristin-skaslien,Kristin SKASLIEN,Norway,NO,NOR
3,Curling,beijing-2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,https://olympics.com/en/athletes/magnus-nedreg...,Magnus NEDREGOTTEN,Norway,NO,NOR
4,Curling,beijing-2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,https://olympics.com/en/athletes/almida-de-val,Almida DE VAL,Sweden,SE,SWE
